<a href="https://colab.research.google.com/github/t8101349/Colab-/blob/master/0114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HTTP代碼

```python
2開頭: 成功

3開頭: 轉址
對面的網址已經搬家了, 雖然妳還是輸入舊網址, 但她會自動幫你替換成新網址

4開頭: 錯誤

404: Not Found
!!! 最常見: 網址打錯
403: Forbidden
!!! 原因1: ip ban
!!! 通常是數次的惡意行為(可能是短時間送太多...)
!!! 解法: 等待/換ip
!!! 原因2: 妳學的不像瀏覽器
!!! 解法: 學的像一點(headers完善)

```

Header

```python
妳在送出網址的時候, 其實不只送出網址
妳送出的是 網址+額外信息(request headers)

妳在得到回應的時候, 其實不是只有得到回應
回應+額外信息(response headers)
```

!!!! 下次檢討

把整頁的的文章的圖片都下載, 並且放在不同資料夾裡
!!! 網址是縮寫的
!!! 有些文章是被刪文的



In [37]:
import urllib.request as req
import bs4
import json
import os

def download_img(url):
  #url = 'https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html'
  r = req.Request(url)
  r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

  # 檔名
  dir_name = url.split('/')[-2] + '/' + url.split('/')[-1]
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)
  #fpath = dir_name + "/" + f_name


  # 如果沒有class的話
  # 1. 找她老爸
  # 2. 自己寫篩選方式(!!!!!)
  allow_subname = ["jpg", "jpeg", "png", "gif"]

  response = req.urlopen(r)
  data = response.read().decode('utf-8') #檔案
  html = bs4.BeautifulSoup(data, 'html.parser')
  links = html.find_all('a')
  for link in links:
      href = link['href']
      #print(link['href'])
      if href.split('.')[-1].lower() in allow_subname:
          #print(href)
          imgurl = href
          r = req.Request(imgurl)
          r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")
          img = req.urlopen(r)
          # open一個新檔案來儲存
          f_name = dir_name + '/' + href.split('/')[-1]
          # 存到資料夾
          # f = open(f_name, 'wb')
          with open(f_name, 'wb') as f:
            f.write(img.read())
          # f.close()

          #print(img.read())
          #print("-" * 30)


In [38]:
def download_page(url):
  r = req.Request(url)
  r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")
  response = req.urlopen(r)
  data = response.read().decode('utf-8') #檔案
  html = bs4.BeautifulSoup(data, 'html.parser')


  posts = html.find_all('div', class_ = 'r-ent')
  for post in posts:
    link = post.find('a')
    #print(link['href'])
    link = 'https://www.ptt.cc' + link['href']
    download_img(link)

download_page('https://www.ptt.cc/bbs/Beauty/index3960.html')

In [ ]:
def download_all_page(url):

In [ ]:
# open
import os
# 如果這個資料夾不存在
# 為何是os.path->下次說
if not os.path.exists("test/test2"):
    # 把它創造出來
    os.makedirs("test/test2")

f = open("test/test2/a.txt", "w", encoding="utf-8")
f.write("abcede")
f.close()

In [32]:
import urllib.request as req
import bs4
import json
import os

url = 'https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html'
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

# 檔名
f_name = url.split('/')[-1]+".json"
dir_name = url.split('/')[-2]

if not os.path.exists(dir_name):
  os.makedirs(dir_name)
fpath = dir_name + "/" + f_name

response = req.urlopen(r)
data = response.read().decode('utf-8') #檔案
html = bs4.BeautifulSoup(data, 'html.parser')

#article = html.find_all('span', class_ = 'article-meta-value')
article = html.find_all('span', {'class' , 'article-meta-value'})
#print(article)
author = article[0].text
if '(' in author:
  authorlist = author.split('(')
  author = authorlist[0]
  nickname = authorlist[1].replace(')', '')
else:
  nickname = ''

board = article[1].text
title = article[2].text
if title.startswith('Re:'):
  title = title.replace('Re:', '')
if title.startswith('['):
  titlelist = title.split(']')
  category = titlelist[0].replace('[', '')
  title = titlelist[1].strip()
else:
  category = ''

date = article[3].text
# 存檔的時候肯定是來個JSON
data = {
    "id":author,
    "nickname":nickname,
    "board":board,
    "category":category,
    "title":title,
    "date":date,
    "push":[]
}

print("作者:", author)
print("當時暱稱:", nickname)
print("看板:", board)
print("分類:", category)
print("標題:", title)
print("時間:", date)

contents = html.find_all('div', {'class' , 'push'})
#table = [] = data['push']
for content in contents:
  push_data = {
      'tag': content.find('span', {'class' , 'push-tag'}).text,
      'user': content.find('span', {'class' , 'push-userid'}).text,
      'content': content.find('span', {'class' , 'push-content'}).text,
      'iptime': content.find('span', {'class' , 'push-ipdatetime'}).text
  }
  data['push'].append(push_data)
print(data['push'])
# Save the data to a JSON file
with open(fpath, 'w', encoding='utf-8') as f:
  json.dump(data, f, ensure_ascii=False, indent=4)



作者: Gentlemon 
當時暱稱: 肥宅紳士
看板: Beauty
分類: 正妹
標題: Cosplay 1601 日本 約兒
時間: Mon Jan 13 16:10:23 2025
[{'tag': '推 ', 'user': 'jht', 'content': ': 大長腿', 'iptime': ' 223.138.43.159 01/13 17:00\n'}, {'tag': '推 ', 'user': 'bio5chris', 'content': ': 好！！！', 'iptime': '223.138.217.143 01/13 20:30\n'}, {'tag': '→ ', 'user': 'bingreen', 'content': ': 長', 'iptime': '  42.77.205.118 01/13 21:13\n'}, {'tag': '推 ', 'user': 'sikadear', 'content': ': 讚', 'iptime': '  36.229.51.166 01/13 21:34\n'}, {'tag': '→ ', 'user': 'sothyengiua', 'content': ': 可', 'iptime': ' 220.132.84.142 01/14 15:53\n'}]


In [ ]:
# 練習: 拿其他的
# 答案
import os
import json
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736733511.A.738.html"
# 順便準備一下等一下要儲存的檔案名稱
url_split = url.split("/")
fname = url_split[-1] + ".json"
dirname = url_split[-2]
# 記得要先創資料夾, 不然紅字
if not os.path.exists(dirname):
    os.makedirs(dirname)
fpath = dirname + "/" + fname

r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

response = req.urlopen(r)
html = bs.BeautifulSoup(response)

meta_list = html.find_all("span", {"class":"article-meta-value"})

ptt_id = meta_list[0].text
if "(" in ptt_id:
    ptt_id_split = ptt_id.split("(")
    ptt_id = ptt_id_split[0].strip()
    nick_name = ptt_id_split[-1].replace(")", "")
else:
    nick_name = ""
board = meta_list[1].text
title = meta_list[2].text
if title[0] == "[":
    title_split = title.split("]")
    category = title_split[0].replace("[", "")
    title = title_split[-1].strip()
else:
    category = ""
date = meta_list[3].text

# 存檔的時候肯定是來個JSON
data = {
    "id":ptt_id,
    "nickname":nick_name,
    "board":board,
    "category":category,
    "title":title,
    "data":date,
    "push":[]
}

print("作者:", ptt_id)
print("當時暱稱:", nick_name)
print("看板:", board)
print("分類:", category)
print("標題:", title)
print("時間:", date)


push_list = html.find_all("div", {"class":"push"})
for push in push_list:
    push_meta = push.find_all("span")
    push_type = push_meta[0].text.strip()
    push_ptt_id = push_meta[1].text.strip()
    push_content = push_meta[2].text.replace(": ", "").strip()
    push_ip_date = push_meta[3].text.strip()
    print(push_type, push_ptt_id, push_content, push_ip_date)
    push_data = {
        "type":push_type,
        "id":push_ptt_id,
        "content":push_content,
        "ipdate":push_ip_date
    }
    data["push"].append(push_data)

# 把json寫入到檔案裡
f = open(fpath, "w", encoding="utf-8")
# 之前是f.write, 現在使用json.dump幫我做更好的write
json.dump(data, f, ensure_ascii=False, indent=4)
f.close()